# Least Squares using Normal Equations
This notebook contains the code for running multiple models on the ML Higgs boson data using the Least Squares with normal equation algorithm.

In [76]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('..')
from helpers import *
from implementations import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Load train and test data

In [27]:
DATA_TRAIN_PATH = '../data/train.csv'
y_train, X_train, ids = load_csv_data(DATA_TRAIN_PATH)

In [28]:
DATA_TEST_PATH = '../data/test.csv'
_, X_test, ids_test = load_csv_data(DATA_TEST_PATH)

In [35]:
X_train.shape

(250000, 30)

## Baseline model using raw data
First let's run the Least Squares algorithm on our raw data without doing any preprocessing. We will use K-fold cross validation to report the metrics on the test data.

In [31]:
# Note that LS with normal equations does not have any hyperparameters to tune
metrics, params = least_squares_cv(y_train, X_train, param_grid={})

In [32]:
metrics, params

({'loss': 0.3397894342676714,
  'accuracy': 74.43039999999999,
  'f1_score': 0.5688070001399301},
 {})

Whoa, not a bad result at all for ordinary Least squares using raw data!

## Baseline model using lightly feature engineered data
Now let's now preprocess our data a bit to handle the missing values (-999) in various ways.

### All features with NaN values imputed
First let's impute all missing values with median of their respective columns. So we will set the `imputable_th` to `1` which means impute all columns with a nan value ratio less than 1, or in other words all columns.

In [77]:
tX_train, ty_train, tX_test, ty_test, cont_features = preprocess(X_train, y_train, X_test, imputable_th=1, encodable_th=0)

In [78]:
tX_train.shape, tX_test.shape

((236483, 31), (568238, 31))

We have now all the columns imputed and plus one more column for the bias.

In [79]:
metrics, params = least_squares_cv(ty_train, tX_train, param_grid={})

In [80]:
metrics, params

({'loss': 0.3256439088814923,
  'accuracy': 76.44240527740189,
  'f1_score': 0.6130156811792764},
 {})

So, we made about 2% improvement on the accuracy and around 5% improvement on the F1 score.

### All features with NaN values encoded
Now let's instead encode these features with NaN values into new indicator features where the new feature takes on a value of 1 if the value for the feature is missing, otherwise 0.

In [70]:
tX_train, ty_train, tX_test, ty_test, cont_features = preprocess(X_train, y_train, X_test, imputable_th=0, encodable_th=1)

In [71]:
tX_train.shape, tX_test.shape

((243430, 31), (568238, 31))

In [72]:
metrics, params = least_squares_cv(ty_train, tX_train, param_grid={})

LinAlgError: Singular matrix

So this results in a `Singular matrix` error which means encoding all columns introduces some linearly dependent columns which should be avoided in order to be used with Least Squares using normal equations. This means the best we can do for least squares is to impute all the columns. Also note that we can not apply a polynomial degree expansion either as those new features are going to be correlated with the original raw features resulting in a similar singular matrix error.

## Baseline model using heavily feature engineered data
In this step, we are going to apply more feature engineering. More specifically, we are going to split our datasets based on the number of jets (`PRI_jet_num`) and create 3 subsets of the data for observations with 0, 1 and more than 1 jet respectively. Each subset will also only have the relevant columns (based on the original paper) All other missing values in the new subsets will be imputed with median values.

In [73]:
X_train_zero, y_train_zero, X_train_one, y_train_one, X_train_many, y_train_many = split_by_jet_num(DATA_TRAIN_PATH, X_train, y_train)
X_test_zero, ids_test_zero, X_test_one, ids_test_one, X_test_many, ids_test_many = split_by_jet_num(DATA_TRAIN_PATH, X_test, ids_test)

In [74]:
X_train_zero.shape, X_train_one.shape, X_train_many.shape

((99913, 15), (77544, 22), (72543, 29))

In [75]:
def train(X_train, y_train, X_test):
    tX_train, ty_train, tX_test, _, cont_features = preprocess(X_train, y_train, X_test, imputable_th=1, encodable_th=0)
    metrics, params = least_squares_cv(ty_train, tX_train, param_grid={})
    return metrics, params

In [81]:
metrics_zero, params_zero = train(X_train_zero, y_train_zero, X_test_zero)

In [82]:
metrics_one, params_one = train(X_train_one, y_train_one, X_test_one)

In [83]:
metrics_many, params_many = train(X_train_many, y_train_many, X_test_many)

In [84]:
metrics_zero, params_zero

({'loss': 0.2642012717831657,
  'accuracy': 81.70239013107171,
  'f1_score': 0.5670434717479905},
 {})

In [85]:
metrics_one, params_one

({'loss': 0.35828878921869395,
  'accuracy': 72.8131634819533,
  'f1_score': 0.5796973376454021},
 {})

In [86]:
metrics_many, params_many

({'loss': 0.3510268171733287,
  'accuracy': 74.3999436897304,
  'f1_score': 0.7144242358678511},
 {})

So, seems like we are doing really good with the data that have zero jets and have on average 76% accuracy now.